### imports

In [1]:
import numpy as np
import sqlite3
import pandas as pd

### Create the Database Vaccinations.db if not exisits

In [2]:
database_name = 'Vaccinations.db'
conn = sqlite3.connect(database_name)
print("Connected to SQLite database Vaccinations.db")
conn.close()

Connected to SQLite database Vaccinations.db


### Create the necessary tables in Vaccinations.db required to store the data of locations.csv in a normalized form

##### Location Table

In [3]:
conn = sqlite3.connect('Vaccinations.db')
cur = conn.cursor()
createLocationsTableSql = """
CREATE TABLE Location (
    location_id INTEGER PRIMARY KEY,
    location TEXT NOT NULL,
    iso_code TEXT NOT NULL
);
"""

try:
    cur.execute(createLocationsTableSql)
    conn.commit()
    print('Successfully Created the table Location')
except Exception as e:
    print(e)
finally:
    conn.close()

Successfully Created the table Location


##### Source Table

In [4]:
conn = sqlite3.connect('Vaccinations.db')
cur = conn.cursor()
createSourceTableSql = """
CREATE TABLE Source (
    source_id INTEGER PRIMARY KEY,
    location_id INTEGER NOT NULL,
    last_observation_date DATE NOT NULL,
    source_name TEXT NOT NULL,
    source_website TEXT NOT NULL,
    FOREIGN KEY (location_id) REFERENCES Location(location_id)
);
"""

try:
    cur.execute(createSourceTableSql)
    conn.commit()
    print('Successfully Created the table Source')
except Exception as e:
    print(e)
finally:
    conn.close()

Successfully Created the table Source


##### Vaccination Table

In [5]:
conn = sqlite3.connect('Vaccinations.db')
cur = conn.cursor()
createVaccinationTableSql = """
CREATE TABLE Vaccination (
    vaccination_id INTEGER PRIMARY KEY,
    location_id INTEGER NOT NULL,
    vaccine TEXT NOT NULL,
    FOREIGN KEY (location_id) REFERENCES Location(location_id)
);
"""

try:
    cur.execute(createVaccinationTableSql)
    conn.commit()
    print('Successfully Created the table Vaccination')
except Exception as e:
    print(e)
finally:
    conn.close()

Successfully Created the table Vaccination


#### Process the CSV File

In [6]:
conn = sqlite3.connect('Vaccinations.db')

# Load your CSV file into a pandas DataFrame
root_df = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/locations.csv')
root_df = root_df.assign(vaccine=root_df['vaccines'].str.split(', ')).explode('vaccine')
root_df.drop('vaccines', axis=1, inplace=True)

# Create the Location dataframe
location_df = root_df[['location', 'iso_code']].drop_duplicates().reset_index(drop=True)
location_df.index.name = 'location_id'
location_df.reset_index(inplace=True)

# Create the Source dataframe
source_df = root_df[['location', 'last_observation_date', 'source_name', 'source_website']].drop_duplicates().reset_index(drop=True)
source_df.index.name = 'source_id'
source_df.reset_index(inplace=True)

source_df = source_df.merge(location_df, on='location', how='left')
source_df.drop('location', axis=1, inplace=True)
source_df.drop('iso_code', axis=1, inplace=True)

# Create the Vaccination dataframe
vaccination_df = root_df[['location', 'vaccine']].drop_duplicates().reset_index(drop=True)
vaccination_df.index.name = 'vaccination_id'
vaccination_df.reset_index(inplace=True)

vaccination_df = vaccination_df.merge(location_df, on='location', how='left')
vaccination_df.drop('location', axis=1, inplace=True)
vaccination_df.drop('iso_code', axis=1, inplace=True)

# Save the dataframes to the database
try:
    print('Trying to save the dataframes to the database')
    location_df.to_sql('Location', conn, if_exists='replace')
    print('Successfully saved the table Location')
    source_df.to_sql('Source', conn, if_exists='replace')
    print('Successfully saved the table Source')
    vaccination_df.to_sql('Vaccination', conn, if_exists='replace')
    print('Successfully saved the table Vaccination')
except Exception as e:
    print(e)
finally:
    conn.close()

Trying to save the dataframes to the database
Successfully saved the table Location
Successfully saved the table Source
Successfully saved the table Vaccination
